In [1]:
import cv2
import os
import sys

## 이미지 저장 폴더

In [2]:
dst_dir = "C:/Users/dlfwj/Desktop/Assualt/optical"

## 원본 동영상 폴더

In [3]:
source_dir = "C:/Users/dlfwj/Desktop/Assualt/Minimize"

In [4]:
source_dir_list = os.listdir(source_dir)
source_dir_list

["11-2_cam01_assault01_place08_night_spring_CUT_00'53_000-00'54_000.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_01'44_000-01'45_000.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_02'31_000-02'32_000.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_02'41_000-02'42_500.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_03'20_000-03'21_000.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_04'20_000-04'21_000.mp4",
 "11-2_cam01_assault01_place08_night_spring_CUT_04'42_000-04'43_000.mp4",
 "11-2_cam02_assault01_place08_night_spring.CUT.00'31-00'43.360p.mpeg4.aac.hdf5",
 "12-1_cam01_assault01_place09_day_spring_CUT_03'02_000-03'03_000.mp4",
 "12-1_cam01_assault01_place09_day_spring_CUT_03'14_000-03'15_000.mp4",
 "12-1_cam01_assault01_place09_day_spring_CUT_03'41_000-03'42_000.mp4",
 "12-1_cam01_assault01_place09_day_spring_CUT_03'58_000-03'59_000.mp4",
 "12-1_cam01_assault01_place09_day_spring_CUT_04'05_000-04'06_000(1).mp4",
 "12-1_cam01_assault01_place09_day_sp

## Optical flow 이미지 저장 (source : https://github.com/deepmind/kinetics-i3d/issues/24)

In [ ]:
import cv2
import pickle
import h5py
import numpy as np
import time
import os
for source_file in source_dir_list:
    if not os.path.exists(dst_dir + "/" + source_file.split(".")[0]):
        os.mkdir(dst_dir + "/" + source_file.split(".")[0]) # 해당 동영상 파일 이름으로 폴더 생성
    cap = cv2.VideoCapture(source_dir+"/"+source_file)  # VideoCapture를 통해 비디오 읽어오기
    flow_list = []
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (256, 256))  # 학습을 위한 이미지 사이즈로 리사이징
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY) # Optical 계산을 위한 GrayScale로 변환
    cnt = 1
    while(1):
        ret, frame2 = cap.read()
        if ret is False:
            break
        start_time = time.time()
        next_frame = cv2.resize(frame2, (256, 256)) # 학습을 위한 이미지 사이즈로 리사이징

        next_frame = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY) # Optical 계산을 위한 GrayScale로 변환
        print(next_frame.shape)

        tvl1 = cv2.DualTVL1OpticalFlow();
        
        flow = tvl1.calc(prvs, next_frame, None); # calculate optical flow 
        flow[flow < -20.0] = -20.0 # i3d 방식의 preprocessing
        flow[flow > 20.0] = 20
        flow = cv2.normalize(flow, None, -1, 1, cv2.NORM_MINMAX)
        
        print(f"time : {time.time() - start_time}")
        cv2.imwrite(dst_dir+"/"+source_file.split(".")[0]+"/flow_x_%04d.jpg" % cnt, flow[:, :, 0]) 
        cv2.imwrite(dst_dir+"/"+source_file.split(".")[0]+"/flow_y_%04d.jpg" % cnt, flow[:, :, 1])
        cnt += 1

        prvs = next_frame

    flow_list = np.array(flow_list, dtype="float32")
    cap.release()



(256, 256)
